<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Turku_ERA5_CDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install "cdsapi>=0.7.7"

In [ ]:
!pip install netcdf4

In [ ]:
import os

# Updated configuration: removed the UID prefix as per the error instructions
content = """url: https://cds.climate.copernicus.eu/api
key: c025f203-5930-4d9c-acd6-699c46be7fd8"""

with open(os.path.expanduser('~/.cdsapirc'), 'w') as f:
    f.write(content)

print("Configuration updated! Now attempting to update the library...")

# Also update your library to the latest version to match the new API
!pip install --upgrade cdsapi

In [ ]:
import cdsapi
import os
import calendar
from tqdm import tqdm # Plain text progress bar (won't break notebook)

c = cdsapi.Client()

# Range: Jan 1 2024 to Jan 1 2025
tasks = [('2024', str(m).zfill(2)) for m in range(1, 13)] + [('2025', '01')]

for year, month in tqdm(tasks, desc="Overall Progress"):
    filename = f'turku_{year}_{month}.nc'

    if os.path.exists(filename):
        continue

    # Monthly day logic
    last_day = calendar.monthrange(int(year), int(month))[1]
    days = ['01'] if (year == '2025' and month == '01') else [str(i).zfill(2) for i in range(1, last_day + 1)]

    try:
        c.retrieve(
            'reanalysis-era5-land',
            {
                'variable': [
                    '2m_temperature',
                    '10m_u_component_of_wind',
                    '10m_v_component_of_wind',
                    'surface_solar_radiation_downwards',
                ],
                'year': year,
                'month': month,
                'day': days,
                'time': [f"{str(i).zfill(2)}:00" for i in range(24)],
                'area': [60.5, 22.5, 60.5, 22.5],
                'format': 'netcdf',
            },
            filename)
    except Exception as e:
        print(f"\n❌ Error on {month}/{year}: {e}")

print("\n✨ All months successfully requested/downloaded.")

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np

def process_turku_data(file_list):
    all_data = []

    for file in file_list:
        ds = xr.open_dataset(file)
        df = ds.to_dataframe().reset_index()
        all_data.append(df)

    # Combine all months
    full_df = pd.concat(all_data)

    # 1. Convert Temperature (Kelvin to Celsius)
    full_df['temp_c'] = full_df['t2m'] - 273.15

    # 2. Calculate Wind Speed (sqrt(u^2 + v^2))
    full_df['wind_speed'] = np.sqrt(full_df['u10']**2 + full_df['v10']**2)

    # 3. Convert GHI (Joules/m2 to Watts/m2)
    # ERA5-Land SSRD is accumulated hourly
    full_df['ghi_w_m2'] = full_df['ssrd'] / 3600

    # Clean up columns
    result = full_df[['valid_time', 'temp_c', 'wind_speed', 'ghi_w_m2']]
    return result

# Example usage (after files download):
# files = ['turku_2024_01.nc', 'turku_2024_02.nc']
# df_final = process_turku_data(files)
# df_final.to_csv('turku_weather_final.csv', index=False)